# Simulations on a TKD periodic domain

The following notebook contains sample code for simulating perfusion and gas exchange on a periodic mesh mimicking a collection of alveoli as a periodic structure based on a tetrakaidecahedron.

First we import necessary libraries and add important directories to the system path.

In [1]:
import sys
import os
import dolfin
import pyvista as pv
sys.path.append(os.getcwd()[:-6])

Following that, we import the main simulation class and parameter dictionary from the model source files.

In [2]:
from src.model import PerfusionGasExchangeModel
from src.params import params

Now we outline the folder name and the path to save raw data files.

In [3]:
folder = "tkd-notebook"
path = os.path.join("../raw-data", folder)

Now we run the solver. First, we instance the model. Then, we import the TKD mesh from an XML file. Afterwards, we solve the perfusion problem on the mesh and then we solve the transport problem _without_ hemoglobin effects. Finally, we run the actual full model using the previous solution as an initial guess for the Newton iteration, and save the boundary data.

In [ ]:
model = PerfusionGasExchangeModel(folder_path=path, params=params)
model.import_mesh(
    os.path.join("../raw-data/meshes", "tkd.xml"), meshtype='tkd', type="xml", 
    periodic=True
)
model.sim_p(save=True, meshtype="tkd")
boundaries = dolfin.File(model.folder_path+'/bnd/bnd.pvd')
boundaries << model.boundaries
x = model.sim_t(hb=False, save=False)
solution = model.sim_t(hb=True, save=True, guess=x)

The preceeding code will generate PVD (Paraview) and VTK files that hold the solutions to our problems. We now visualize the results from a parallel simulation (see the `cluster` directory for details on how to run in a cluster).

In [ ]:
mesh = pv.read(os.path.join('../raw-data','tkd-job','t','pO2.pvd'))
mesh.plot(jupyter_backend='pythreejs')